***NOTES:***

In [2]:
print('\U0001F917')
open()

🤗


Invest in each portfolio in a manner proportional to its individual expected return divided by its individual risk.
`Watchlist`
* FinVolution Group (NASDAQ:FINV)
* Himax Technologies (NASDAQ:HIMX)
* iBio (NASDAQ:IBIO)
* Kopin Corp (NASDAQ:KOPN)
* Ribbon Communications (NASDAQ:RBBN)
* StoneMor (NASDAQ:STON)

`IPOs:`
* Instacart
* Robinhood
* Coupang

`Short Squeeze Rubric:`
- High short volume / % float
- Large Put volume open interest near current stock price.
- Then buy a portfolio of slightly OTM calls with 2-3 weeks to expiry, and roll the portfolio weekly on Wednesday.

In [1]:
import eikon as ek
# import cufflinks as cf
import tensorflow as tf
import tensorflow.keras
# from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K
from tqdm import tqdm
import os
import time
import pandas as pd
import numpy as np
import psutil
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from pathlib import Path
# My Custom Functions
from functions import *
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.option_context('display.max_rows', 1000)

In [2]:
#display
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [4]:
ek.set_app_key('b3128062087743ed9ccc1b11527f589a9841f6fa')
ek.get_port_number()

'9000'

In [14]:
ek .get_data('USD3MFSR=X',['TR.FIXINGVALUE.date','TR.FIXINGVALUE'],{"Sdate":'2005-01-03', "Edate":'2019-01-03'})

(      Instrument                  Date  Fixing Value
 0     USD3MFSR=X  2005-01-03T00:00:00Z           NaN
 1     USD3MFSR=X  2005-01-04T00:00:00Z       2.57000
 2     USD3MFSR=X  2005-01-05T00:00:00Z       2.59000
 3     USD3MFSR=X  2005-01-06T00:00:00Z       2.61000
 4     USD3MFSR=X  2005-01-07T00:00:00Z       2.61000
 ...          ...                   ...           ...
 3535  USD3MFSR=X  2018-12-27T00:00:00Z       2.80300
 3536  USD3MFSR=X  2018-12-28T00:00:00Z       2.79700
 3537  USD3MFSR=X  2018-12-31T00:00:00Z       2.80763
 3538  USD3MFSR=X  2019-01-02T00:00:00Z       2.79388
 3539  USD3MFSR=X  2019-01-03T00:00:00Z       2.79500
 
 [3540 rows x 3 columns],
 None)

## STOCKS PORTFOLIO
* Invest in each portfolio in a manner proportional to its individual expected return divided by its individual risk.
* Although bet size should increase with expected return, when the signal is very large don’t bet the whole farm. Bet about a third of the farm.

In [5]:
ratings_data, err = ek.get_data(instruments=['RKT', 'GME', 'LIVE.O', 'AMC.N', 'NVAX.O', 'IBIO.K', 'OCGN.O','MNOV.O',
                                             'FINV.K', 'VXRT.O', 'JAGX.O', 'XAU=', 'VIXM.K', 'EScv1', 
                                             'NQcv1', '.INX', '.VIX',
                                             'US10YT=RR',
                                             'USDAM3L10Y='], 
                                fields = ['TR.CommonName',
                                        'PRCTCK_1',
                                        'TR.SharesFreeFloat',
                                        'TR.ShortInterest',
                                        'TR.Volume',
                                        'TR.VWAP',
                                        'BID',
                                        'ASK',
                                        'LAST',
                                        'TR.IssuerRating(IssuerRatingSrc=SPI)',
                                        'TR.PE',
                                         'TR.InsiderBuyDepthComp',
                                         'TR.WACCBeta'])
ratings_data['Short Interest(%)'] = 100 * (ratings_data['Short Interest'] / ratings_data['Free Float'])
ratings_data['BID-ASK Spread'] = ratings_data['ASK'] - ratings_data['BID']
if sum(ratings_data.LAST.isna()) == len(ratings_data):
    ratings_data['LAST'] = (ratings_data['ASK'] + ratings_data['BID']) / 2
ratings_data.set_index('Instrument', inplace = True)
ratings_data

Company Common Name PRCTCK_1   Free Float  Short Interest      Volume  Volume Weighted Average Price           BID        ASK          LAST Issuer Rating  P/E (Daily Time Series Ratio)  Insider Buying Depth Component      Beta  Short Interest(%)  BID-ASK Spread
Instrument                                                                                                                                                                                                                                                                                          
RKT                           Rocket Companies Inc        ⇩  113143393.0      39656989.0  12200184.0                      23.344184     23.310000     23.350     23.330000                                    14.111041                             NaN  1.385846          35.050203        0.040000
GME                                  GameStop Corp        ⇩   54946471.0      14200138.0  24677297.0                     197.046382    198.230000    198.470    198.350000            B-                            NaN                            91.0 -1.893644          25.843585        0.240000
LIVE.O                           Live Ventures Inc        ⇧    1291094.0          6745.0      4783.0                      23.320000     22.000000     28.000     25.000000                                     5.603851                             NaN  1.141936           0.522425        6.000000
AMC.N               AMC Entertainment Holdings Inc        ⇧  403836464.0      55490440.0  55222025.0                            NaN           NaN        NaN           NaN          CCC-                            NaN                             1.0  1.461163          13.740820             NaN
NVAX.O                                 Novavax Inc        ⇩   72400525.0       5332186.0   2864593.0                     227.138171    226.800000    227.750    227.275000                                          NaN                            54.0  1.553182           7.364844        0.950000
IBIO.K                                    Ibio Inc        ⇩  214097322.0      10234416.0  11456705.0                       1.730724      1.720000      1.740      1.730000                                          NaN                             1.0 -6.229844           4.780263        0.020000
OCGN.O                                  Ocugen Inc        ⇧  182700876.0      12432667.0  24255773.0                       8.853558      8.850000      8.890      8.870000                                          NaN                            94.0  4.400812           6.804930        0.040000
MNOV.O                              MediciNova Inc        ⇧   45126879.0       2309945.0    483411.0                       5.802878      5.800000      5.820      5.810000                                          NaN                            23.0  1.494925           5.118779        0.020000
FINV.K                           FinVolution Group        ⇧  149127969.0        683813.0   4592793.0                       8.179780      8.200000      8.230      8.215000                                     8.572999                             NaN  1.004235           0.458541        0.030000
VXRT.O                                  Vaxart Inc        ⇩  117056513.0      26127439.0   6682371.0                       6.877091      6.910000      6.940      6.925000            NR                            NaN                            65.0  0.104203          22.320363        0.030000
JAGX.O                           Jaguar Health Inc        ⇩  105332181.0       3392971.0   7214287.0                       2.148717      2.150000      2.170      2.160000                                          NaN                            74.0  1.613687           3.221210        0.020000
XAU=                                          None     None          NaN             NaN         NaN                            NaN   1734.120000   1734.630   1734.375000          None                            NaN       

## OPTIONS PORTFOLIO

In [30]:

df, err = ek.get_data(
    instruments = ['IBIOC192100250.U', 'SOLOC052100700.U', 'VXRTC052101000.U', 'OCGNC192102000.U','FINVC192101250.U', 'GSATC192100500.U'],
    fields = [
        'STOCK_RIC',
        'LAST',
        'BID',
        'ASK',
        'PCTCHNG',
        'DSPLY_NAME',
        'NETCHNG_1',
        'OPINT_1',
        'PUT_CALL',
        'STRIKE_PRC'
    ]
)
df
# options['Underlying'] = options['Instrument'].str.split('C')[0]


Instrument STOCK_RIC  LAST  BID  ASK  PCTCHNG        DSPLY_NAME  NETCHNG_1  OPINT_1 PUT_CALL  STRIKE_PRC
0  IBIOC192100250.U    IBIO.K   NaN  NaN  NaN      NaN  IBIO Mar1 2.5 /d        NaN  12497.0     C-AM         2.5
1  SOLOC052100700.U      None   NaN  NaN  NaN      NaN              None        NaN      NaN     None         NaN
2  VXRTC052101000.U      None   NaN  NaN  NaN      NaN              None        NaN      NaN     None         NaN
3  OCGNC192102000.U    OCGN.O   NaN  NaN  NaN      NaN  OCGN Mar1 20.0/d        NaN  10412.0     C-AM        20.0
4  FINVC192101250.U    FINV.K   NaN  NaN  NaN      NaN  FINV Mar1 12.5/d        NaN   3164.0     C-AM        12.5
5  GSATC192100500.U    GSAT.K   NaN  NaN  NaN      NaN  GSAT Mar1 5.0 /d        NaN   8006.0     C-AM         5.0

In [3]:
import requests
premarket_gainers = requests.get('https://api.apispreadsheets.com/data/6016/')
premarket_gainers = pd.DataFrame(premarket_gainers.json()['data'])
premarket_losers = requests.get('https://api.apispreadsheets.com/data/6018/')
premarket_losers = pd.DataFrame(premarket_losers.json()['data'])

In [4]:
premarket_gainers

ticker exchange 2021-04-09 2021-04-08 2021-04-07 2021-04-06 2021-04-05        MC  volume                             urls
0    GLSI       US     -15.09     -17.58     -18.05      -7.54       7.06    688.59       6   https://sycnal.co/?symbol=GLSI
1    AFMD       US      -5.26      -6.02      -8.03     -13.63      -7.55   1142.37   69.87   https://sycnal.co/?symbol=AFMD
2     ASO       US      -0.48      -2.04      -4.54     -10.38      -8.88   2815.58    5.26    https://sycnal.co/?symbol=ASO
3    FUBO       US       5.13       4.77      -0.79        3.3      -3.08   3267.15  103.12   https://sycnal.co/?symbol=FUBO
4     DAN       US       1.81       2.26      -0.03       1.88       0.22   3747.03    1.46    https://sycnal.co/?symbol=DAN
5    BGCP       US       5.37       6.88       3.08       9.06      11.21   2103.03    4.93   https://sycnal.co/?symbol=BGCP
6    OSTK       US       5.73       9.87        3.2        5.4       -0.9   3083.06    2.92   https://sycnal.co/?symbol=OSTK
7     FND       US       4.33       4.75       5.13       7.71       7.22  11370.06    1.97    https://sycnal.co/?symbol=FND
8     PVH       US        5.1       7.07       5.19       5.53       7.37   7864.33    1.42    https://sycnal.co/?symbol=PVH
9    COMM       US       5.42       6.53       5.31       5.31       5.31   3559.47    8.31   https://sycnal.co/?symbol=COMM
10  SRNGU       US       5.54       5.93       5.64       5.93       5.73      2208   14.92  https://sycnal.co/?symbol=SRNGU
11   HOME       US       6.05       9.22       7.12       7.87        4.4    1879.6    1.16   https://sycnal.co/?symbol=HOME
12   DASH       US      11.23      10.44       7.49       6.93       5.85  45165.61    6.48   https://sycnal.co/?symbol=DASH
13   CARA       US       6.19       9.97       8.41       10.4      17.98    1455.9    2.07   https://sycnal.co/?symbol=CARA
14   LOVE       US      12.11      16.44      12.81      16.22      16.77   1053.51    1.33   https://sycnal.co/?symbol=LOVE
15   STPK       US       5.67      18.21      16.36       16.4      10.21    1493.1    3.38   https://sycnal.co/?symbol=STPK
16   FBIO       US       6.05      18.75      22.55      20.19      22.15     516.3   21.62   https://sycnal.co/?symbol=FBIO
17   RICE       US      18.99      27.04      26.55      26.75      25.38     513.1    22.1   https://sycnal.co/?symbol=RICE
18   REPX       US      25.89      31.86      27.62      25.38      28.83    575.79    1.09   https://sycnal.co/?symbol=REPX
19   VECT       US        NaN        NaN        NaN        NaN        NaN    631.76    5.28   https://sycnal.co/?symbol=VECT

In [5]:
premarket_losers

ticker exchange 2021-04-09 2021-04-08 2021-04-07 2021-04-06 2021-04-05        MC  volume                             urls
0    REPX       US      25.89      31.86      27.62      25.38      28.83    575.79    1.09   https://sycnal.co/?symbol=REPX
1    RICE       US      18.99      27.04      26.55      26.75      25.38     513.1    22.1   https://sycnal.co/?symbol=RICE
2    FBIO       US       6.05      18.75      22.55      20.19      22.15     516.3   21.62   https://sycnal.co/?symbol=FBIO
3    STPK       US       5.67      18.21      16.36       16.4      10.21    1493.1    3.38   https://sycnal.co/?symbol=STPK
4    LOVE       US      12.11      16.44      12.81      16.22      16.77   1053.51    1.33   https://sycnal.co/?symbol=LOVE
5    CARA       US       6.19       9.97       8.41       10.4      17.98    1455.9    2.07   https://sycnal.co/?symbol=CARA
6    DASH       US      11.23      10.44       7.49       6.93       5.85  45165.61    6.48   https://sycnal.co/?symbol=DASH
7    HOME       US       6.05       9.22       7.12       7.87        4.4    1879.6    1.16   https://sycnal.co/?symbol=HOME
8   SRNGU       US       5.54       5.93       5.64       5.93       5.73      2208   14.92  https://sycnal.co/?symbol=SRNGU
9    COMM       US       5.42       6.53       5.31       5.31       5.31   3559.47    8.31   https://sycnal.co/?symbol=COMM
10    PVH       US        5.1       7.07       5.19       5.53       7.37   7864.33    1.42    https://sycnal.co/?symbol=PVH
11    FND       US       4.33       4.75       5.13       7.71       7.22  11370.06    1.97    https://sycnal.co/?symbol=FND
12   OSTK       US       5.73       9.87        3.2        5.4       -0.9   3083.06    2.92   https://sycnal.co/?symbol=OSTK
13   BGCP       US       5.37       6.88       3.08       9.06      11.21   2103.03    4.93   https://sycnal.co/?symbol=BGCP
14    DAN       US       1.81       2.26      -0.03       1.88       0.22   3747.03    1.46    https://sycnal.co/?symbol=DAN
15   FUBO       US       5.13       4.77      -0.79        3.3      -3.08   3267.15  103.12   https://sycnal.co/?symbol=FUBO
16    ASO       US      -0.48      -2.04      -4.54     -10.38      -8.88   2815.58    5.26    https://sycnal.co/?symbol=ASO
17   AFMD       US      -5.26      -6.02      -8.03     -13.63      -7.55   1142.37   69.87   https://sycnal.co/?symbol=AFMD
18   GLSI       US     -15.09     -17.58     -18.05      -7.54       7.06    688.59       6   https://sycnal.co/?symbol=GLSI
19   VECT       US        NaN        NaN        NaN        NaN        NaN    631.76    5.28   https://sycnal.co/?symbol=VECT

## News Headlines Model Predictions:

In [2]:
from pathlib import Path
DATA_PATH = 'models_DB.h5'
news_db = "news_db.h5"

In [11]:
with pd.HDFStore(news_db, mode = 'r') as store:
    df = store["A"]
    store.put()

In [12]:
df

versionCreated                                               text                                       storyId sourceCode ticker
2020-09-01 21:20:54.464 2020-09-01 21:20:54.464000+00:00  AGILENT TECHNOLOGIES, INC. SEC Filings files F...  urn:newsml:reuters.com:20200901:nEOL6H1s4Y:1     NS:EDG      A
2020-09-01 20:43:42.083 2020-09-01 20:43:42.083000+00:00  AGILENT TECHNOLOGIES, INC. Files 10-Q for Quar...  urn:newsml:reuters.com:20200901:nEOL2MG35F:1     NS:EDG      A
2020-09-01 12:00:01.815 2020-09-01 12:00:01.815000+00:00  Agilent Announces SureSelect XT HS2 RNA Reagen...  urn:newsml:reuters.com:20200901:nBw2WycKya:1     NS:BSW      A
2020-09-02 12:15:51.215 2020-09-02 12:15:51.215000+00:00  Agilent Announces SureSelect XT HS2 RNA Reagen...  urn:newsml:reuters.com:20200902:nNRActyfem:1  NS:ENPNWS      A
2020-09-03 19:51:17.723 2020-09-03 19:51:17.723000+00:00  NYSE ORDER IMBALANCE <A.N> 97859.0 SHARES ON S...  urn:newsml:reuters.com:20200903:nAQN031W47:1    NS:RTRS      A
...                                                  ...                                                ...                                           ...        ...    ...
2021-07-20 16:14:58.000        2021-07-20 16:14:58+00:00  Agilent Technologies Inc. - Agilent PD-L1 IHC ...  urn:newsml:reuters.com:20210720:nNDL8wVTcK:1    NS:PUBT      A
2021-07-20 12:44:03.051 2021-07-20 12:44:03.051000+00:00  Thomas Scientific Lands Agilent as Distributio...  urn:newsml:reuters.com:20210720:nPn3bnltLa:1     NS:PRN      A
2021-07-20 12:30:01.324 2021-07-20 12:30:01.324000+00:00  Agilent Announces Optimized and Verified Metho...  urn:newsml:reuters.com:20210720:nBw7NRLxma:1     NS:BSW      A
2021-07-20 12:21:50.000        2021-07-20 12:21:50+00:00  Agilent Technologies Inc. - Agilent Announces ...  urn:newsml:reuters.com:20210720:nNDL4P8dfv:1    NS:PUBT      A
2021-07-20 11:29:45.122 2021-07-20 11:29:45.122000+00:00  AGILENT TECHNOLOGIES INC <A.N>: SVB LEERINK RA...  urn:newsml:reuters.com:20210720:nFWN2OW0PB:1    NS:RTRS      A

[1735 rows x 5 columns]

In [9]:
# from Scripts.daily_model_predict import predict_news
# predict_news()
# export TF_XLA_FLAGS=--tf_xla_cpu_global_jit
os.environ['TF_XLA_FLAGS'] = "--tf_xla_cpu_global_jit"

#### Summary Stat of News Headlines DB:

In [10]:
length = 0
max_dates = []
min_dates = []
#added to fix broken tables in the HDF5 file
broken_tables = []
news_per_ticker = {}
with pd.HDFStore(news_db, mode = 'r') as store:
    for i in tqdm(store.keys()):
        if isinstance(store[i], pd.DataFrame):
            max_dates.append(store[i].versionCreated.max())
            min_dates.append(store[i].versionCreated.min())
            news_per_ticker[str(i)] = len(store[i])
            length += len(store[i])
        else:
            #add tickers to list
            broken_tables.append(i)
            pass
print('News Headlines available: ', length)
print('Min date found: ', min(min_dates))
print('Max date found: ', max(max_dates))
print('Average News Headlines per Company: ', int(length/530))
print('Ticker with Max News headlines: ', max(news_per_ticker, key=news_per_ticker.get))
print('Ticker with Min News headlines: ', min(news_per_ticker, key=news_per_ticker.get))

100%|████████████████████████████████████████████████████████████████████████████████| 549/549 [02:56<00:00,  3.12it/s]

News Headlines available:  1529859
Min date found:  2019-07-08 07:20:18.185000+00:00
Max date found:  2021-07-16 22:13:41.943000+00:00
Average News Headlines per Company:  2886
Ticker with Max News headlines:  /IVZ
Ticker with Min News headlines:  /AIRC


In [4]:
with pd.HDFStore(DATA_PATH, mode = 'r+') as store:
    print(store.keys()) 
    df = store['/prices/daily']

['/prices/daily', '/predictions/news/daily', '/predictions/news/true_daily']


In [5]:
df

open   close    high     low      volume
date       ticker                                            
2021-01-04 A       118.94  118.64  120.06  117.11    684690.0
           AAL      15.86   15.13   15.87   15.03  10712068.0
           AAP     157.90  157.34  159.53  156.01    253147.0
           AAPL    133.56  129.41  133.60  126.76  35418103.0
           ABBV    107.18  105.41  107.28  103.86   2255821.0
...                   ...     ...     ...     ...         ...
2021-07-20 ZION     48.81   50.20   51.61   48.06   1562935.0
2021-07-19 ZM      356.69  354.20  357.30  343.82   2164650.0
2021-07-20 ZM      357.82  355.81  359.14  347.08   1235649.0
2021-07-19 ZTS     198.47  197.83  199.48  195.70    596146.0
2021-07-20 ZTS     198.10  200.11  201.45  197.79    576047.0

[66177 rows x 5 columns]

In [8]:
# with pd.HDFStore('out.h5') as f:
#     #delete broken table
#     del f['/KSU']
#     f.close()

#### Get News By Date

In [2]:
with pd.HDFStore('out.h5', mode = 'r') as store:
    final_df = pd.DataFrame()
    for i in tqdm(store.keys()):
        df = store[i]
        yesterday = (datetime.datetime.now() +datetime.timedelta(-1)).strftime('%Y-%m-%d')
#        if df.loc[:, 'versionCreated'].apply(lambda x: x.strftime('%Y-%m-%d')) == yesterday:
        news_yesterday = df.loc[df.loc[:, 'versionCreated'].apply(lambda x: x.strftime('%Y-%m-%d')) == yesterday]
        news_yesterday = news_yesterday[~news_yesterday.text.duplicated()]
        if len(news_yesterday) > 0:
            sample = news_yesterday.text.str.cat(sep=' ')
            if len(sample) > 1000: sample = sample[:500] + "..." + sample[-500:]
            else: pass   
            temp_df = pd.DataFrame.from_dict({i.strip('/'): sample}, orient='index').rename(columns={0:'Headlines'})
            final_df = final_df.append(temp_df)
            final_df['Date'] = yesterday
            final_df = final_df[~final_df.Headlines.duplicated()]
        else: pass

100%|████████████████████████████████████████████████████████████████████████████████| 548/548 [01:38<00:00,  5.58it/s]


In [3]:
key_words = ['NYSE ORDER IMBALANCE','clinial trials', 'inflation', 'treasury', 'yields', 'wall street bets', 'wallstreetbets']
for i in key_words:
    if len(final_df[final_df['Headlines'].str.contains(i)]) >= 1:
        df = final_df[final_df['Headlines'].str.contains(i)]
        final_df[final_df['Headlines'].str.contains(i)].to_csv('%s.csv' % i)
        print(f'Found {i}')

Found NYSE ORDER IMBALANCE
Found inflation


#### Check Predictions Database

In [8]:
with pd.HDFStore(DATA_PATH) as store:
    for i in store.keys():
        print(i,":  Max Found Date -->",store[i].index.get_level_values(0).max().strftime('%Y-%m-%d'))
    #check max date of prediction to run or kill
    df = store['predictions/news/daily'].sort_index()
    max_date = df.index.get_level_values(0).max().strftime('%Y-%m-%d')
    today = (datetime.datetime.now()).strftime('%Y-%m-%d') # add timedelta(1) if running previous day
    if max_date == today:
        df = df.loc[today]
    else: print('No Prediction found for %s' % today)

/prices/daily :  Max Found Date --> 2021-03-29
/predictions/news/daily :  Max Found Date --> 2021-03-30
/predictions/news/true_daily :  Max Found Date --> 2021-03-29


In [15]:
df.loc[(slice(None),'AFL'),]

Headlines  News Date  Predictions_V1  Predictions_V2  BUY_V1  BUY_V2
date       ticker                                                                                                              
2021-03-30 AFL     NYSE ORDER IMBALANCE <AFL.N> 59887.0 SHARES ON... 2021-03-29         0.11652         0.04129       0       0

In [13]:
df[df.BUY_V1 != df.BUY_V2]
# df[df.index == 'ZM']['Headlines'][0]

Headlines  News Date  Predictions_V1  Predictions_V2  BUY_V1  BUY_V2
date       ticker                                                                                                              
2021-03-30 ACN     Accenture to Acquire Cygni Accenture consultan... 2021-03-29         0.20630         0.53930       0       1
           ADI     ANALOG DEVICES DIPS IN VOLUME SPIKE, LAST DOWN... 2021-03-29         1.00000         0.03847       1       0
           ADSK                          Autodesk, Inc. Files -- 8-K 2021-03-29         0.07162         0.96699       0       1
           AMT                 AMERICAN TOWER CORP /MA/ Files -- 8-K 2021-03-29         1.00000         0.04180       1       0
           ATVI    Activision Blizzard, Inc (ATVI) Gains As Marke... 2021-03-29         0.24527         0.53162       0       1
           CME     CME GROUP - MORE THAN 1 MLN OPTIONS ON ITS MIC... 2021-03-29         0.12698         0.57801       0       1
           DISCA   US STOCKS-S&P 500 ends a hair lower; hedge fun... 2021-03-29         0.16639         0.55652       0       1
           DISH                         DISH Network CORP -- UNKNOWN 2021-03-29         1.00000         0.34248       1       0
           HFC                       HollyFrontier Corp Files -- 8-K 2021-03-29         0.07721         0.97532       0       1
           HON     U.S. RESEARCH ROUNDUP- CytomX Therapeutics, No... 2021-03-29         0.36051         0.65194       0       1
           MKC     McCormick Canada Announces Multi-Million Dolla... 2021-03-29         0.25437         0.54810       0       1
           PYPL             Now on CNBC LIVE US: PayPal Holdings Inc 2021-03-29         0.19689         0.70716       0       1
           QCOM    UPDATE 1-U.S. trade regulator will not appeal ... 2021-03-29         0.38472         0.53522       0       1
           RCL                    ROYAL CARIBBEAN CRUISES LTD -- 4/A 2021-03-29         1.00000         0.11026       1       0
           SNPS             SYNOPSYS INC SEC Filings files Form -- 4 2021-03-29         0.53845         0.09448       1       0
           TT         CMC Cielo hires Investor Cubed for IR services 2021-03-29         0.17023         0.75840       0       1
           TTWO         TAKE TWO INTERACTIVE SOFTWARE INC -- UNKNOWN 2021-03-29         1.00000         0.15218       1       0
           VRSN          VERISIGN INC/CA SEC Filings files Form -- 4 2021-03-29         0.59943         0.02208       1       0
           WAB     Fitch Affirms Wabtec at 'BBB-'; Outlook Remain... 2021-03-29         0.30950         0.52360       0       1

### Options Analytics:

In [83]:
import refinitiv.dataplatform as rdp
from refinitiv.dataplatform.content import ipa
from refinitiv.dataplatform.content.ipa import bond
from refinitiv.dataplatform.content.ipa import option
import datetime
import asyncio

rdp.open_desktop_session('b3128062087743ed9ccc1b11527f589a9841f6fa')

In [84]:
def display_response(response):
    data_type = response.closure
    current_time = datetime.datetime.now().time()
    display('{} received at {}'.format(data_type, current_time))
    display(response.data.df)

In [103]:
asyncio.get_event_loop().create_task(
    ipa.FinancialContracts.get_option_analytics_async(
        universe = ['NVAXB262132000.U', "IBIOC192100250.U", "SOLOC052100700.U", 'VXRTC052101000.U', 'OCGNC192102000.U', 'FINVC192101250.U', 'GSATC192100500.U'],
        on_response = lambda params, response : display_response(response))
)

asyncio.get_event_loop().create_task(
    ipa.FinancialContracts.get_instrument_analytics_async(
        [
            option.Definition("NVAXB262132000.U"),
            option.Definition("IBIOC192100250.U")
        ]
    )
)

<Task pending coro=<FinancialContracts.get_instrument_analytics_async() running at c:\users\firo obeid\anaconda3\envs\pushstashenv\lib\site-packages\refinitiv\dataplatform\content\ipa\contracts\_financial_contracts.py:132>>

'None received at 12:56:44.581567'

InstrumentTag             InstrumentDescription         ValuationDate    InstrumentCode               EndDate  StrikePrice OptionType ExerciseStyle ExerciseType OptionPriceSide OptionTimeStamp  OptionPrice OptionCcy UnderlyingRIC UnderlyingPriceSide UnderlyingTimeStamp  UnderlyingPrice UnderlyingCcy     DividendType  DividendYieldPercent  MarketValueInDealCcy VolatilityType  VolatilityPercent  RiskFreeRatePercent  DeltaPercent  GammaPercent  RhoPercent  ThetaPercent  VegaPercent  GearingInDealCcy  IntrinsicInDealCcy  TimeValueInDealCcy  BreakEvenTimeInDealCcy  VannaInDealCcy  VolgaInDealCcy  SpeedInDealCcy  CharmInDealCcy  ColorInDealCcy  DeltaInDealCcy  GammaInDealCcy  RhoInDealCcy  ThetaInDealCcy  VegaInDealCcy                                 DiscountCurveId                             DividendCurveId ErrorCode ErrorMessage ProcessingInformation
0          None  CashOption_AMER_NVAXB262132000.U  2021-02-24T00:00:00Z  NVAXB262132000.U  2021-02-26T00:00:00Z        320.0    Vanilla          AMER         CALL            Last         Default         0.80       USD        NVAX.O                Last             Default         238.2300           USD  HistoricalYield              0.000000                  0.80        Implied         230.526767             0.147525      0.050101      0.002541    0.000610     -1.049911     0.018217        297.787500                   0                0.80                2.281663        0.081311        2.357119        0.000092       17.105053        0.459808        5.010098        0.254110      0.061017     -104.991100       1.821682  IRCurve_USD-SwapSB/3MLibor_2021-02-24T00:00:00  HistoricalYield_NVAX.O_0001-01-01T00:00:00                                             
1          None  CashOption_AMER_IBIOC192100250.U  2021-02-24T00:00:00Z  IBIOC192100250.U  2021-03-19T00:00:00Z          2.5    Vanilla          AMER         CALL            Last         Default         0.19       USD        IBIO.K                Last             Default           2.0415           USD  HistoricalYield              0.000000                  0.19        Implied         166.900564             0.160981      0.392093      0.449261    0.000385     -0.007147     0.001969         10.744737                   0                0.19               61.028997        0.159512        0.022387       -0.076213        2.113911       -2.887922       39.209334       44.926090      0.038467       -0.714748       0.196920  IRCurve_USD-SwapSB/3MLibor_2021-02-24T00:00:00  HistoricalYield_IBIO.K_0001-01-01T00:00:00                                             
2          None  CashOption_AMER_SOLOC052100700.U  2021-02-24T00:00:00Z  SOLOC052100700.U  2021-03-05T00:00:00Z          7.0    Vanilla          AMER         CALL            Last         Default         0.41       USD        SOLO.O                Last             Default           6.6950           USD  HistoricalYield              0.000000                  0.41        Implied         128.214975             0.149976      0.452076      0.293831    0.000645     -0.029670     0.004164         16.329268                   0                0.41               41.517200        0.099390        0.012582       -0.017638        2.587052       -5.727113       45.207550       29.383066      0.064520       -2.966967       0.416377  IRCurve_USD-SwapSB/3MLibor_2021-02-24T00:00:00  HistoricalYield_SOLO.O_0001-01-01T00:00:00                                             
3          None  CashOption_AMER_VXRTC052101000.U  2021-02-24T00:00:00Z  VXRTC052101000.U  2021-03-05T00:00:00Z         10.0    Vanilla          AMER         CALL            Last         Default         0.10       USD        VXRT.O                Last             Default           7.1700           USD  HistoricalYield              0.000000                  0.10        Implied         163.795006             0.149976      0.122069      0.109786    0.000191     -0.020746     0.002279         71.700000                   0                0.10           

In [ ]:
rdp.close_session()